In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.extensions import UnitaryGate
from qiskit.quantum_info import Statevector
from qiskit.circuit.library import MCXGate

In [2]:
def reflection_operator(n):
    qc1 = QuantumCircuit(n)
    qc1.h(0)
    qc1.append(MCXGate(n-1), list(range(1, n))+[0])
    qc1.h(0)
    return qc1.to_gate()

In [3]:
n = 3
N = 2**n

In [4]:
oracle_matrix = np.diag([1, 1, 1, 1, 1, 1, -1, 1])
oracle_gate = UnitaryGate(oracle_matrix)

In [6]:
qc = QuantumCircuit(n)
qc.h(range(n))
qc.append(oracle_gate, range(n))
qc.append(reflection_operator(n), range(n))
qc.h(range(n))
Statevector(qc)

Statevector([9.52420783e-18+0.j, 1.00000000e+00+0.j, 1.26316153e-34+0.j,
             2.36158002e-17+0.j],
            dims=(2, 2))


In [5]:
Statevector(qc)

Statevector([ 0.5+0.j, -0.5+0.j,  0.5+0.j,  0.5+0.j],
            dims=(2, 2))
